In [5]:
import numpy as np
import pandas as pd

In [6]:
!pip install pyLDAvis

In [7]:
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [8]:
!pip install --upgrade pythainlp

In [9]:
import pythainlp

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [171]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Customer Analytics/Voice of Customer/Momo_Reviews.csv')

In [172]:
df.tail()

,ReviewID,Review
25,cust26,This branch is close to home to eat regularly....
26,cust27,good service staff no need to wait long for food
27,cust28,"Good quality meat, delicious sauce, good servi..."
28,cust29,A lot of people but not crowded
29,cust30,Good food and service. But it's too crowded. N...


#Tokenize


In [173]:
stopwords = list(pythainlp.corpus.thai_stopwords())

In [174]:
import nltk
# Download the NLTK stopwords dataset (if not already downloaded)
nltk.download("stopwords")

# Import the English stopwords from NLTK
from nltk.corpus import stopwords

# Get the list of English stopwords
english_stopwords = stopwords.words("english")

# Print the list of English stopwords
print(english_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [175]:
removed_words = ['',' ','  ','\n', '(', ')','-','=',' ','/','.','\'','!',"a", "about", "above", "after", "again", "against", "ain't", "all", "am", "an", "and", "any", "are", "aren't",
    "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can't",
    "cannot", "could", "couldn't", "did", "didn't", "do", "does", "doesn't", "doing", "don't", "down", "during",
    "each", "few", "for", "from", "further", "had", "hadn't", "has", "hasn't", "have", "haven't", "having", "he",
    "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i",
    "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "isn't", "it", "it's", "its", "itself", "let's", "me",
    "more", "most", "mustn't", "my", "myself", "no", "nor", "not", "of", "off", "on", "once", "only", "or", "other",
    "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "shan't", "she", "she'd", "she'll", "she's",
    "should", "shouldn't", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them",
    "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this",
    "those", "through", "to", "too", "under", "until", "up", "very", "was", "wasn't", "we", "we'd", "we'll",
    "we're", "we've", "were", "weren't", "what", "what's", "when", "when's", "where", "where's", "which", "while",
    "who", "who's", "whom", "why", "why's", "with", "won't", "would", "wouldn't", "you", "you'd", "you'll", "you're",
    "you've", "your", "yours", "yourself", "yourselves","The","But"]

In [177]:
screening_words = english_stopwords + removed_words

In [178]:
def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [181]:
df['Comment_token'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [182]:
df.head()

,ReviewID,Review,Comment_token
0,cust01,It's very delicious. Definitely will come agai...,"It,delicious,Definitely,come,black,soup,great,..."
1,cust02,delicious !,delicious
2,cust03,"During Saturday and Sunday, there are many peo...","During,Saturday,Sunday,,,many,people,waiting,line"
3,cust04,Food is great with lots of options in vegetabl...,"Food,great,lots,options,vegetables,sauces"
4,cust05,"Good service staff, Sa-ard restaurant, delicio...","Good,service,staff,,,Sa-ard,restaurant,,,delic..."


#CREATE DICTIONNARY

In [183]:
documents = df['Comment_token'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [184]:
print(dictionary.token2id.keys())

dict_keys(['3', 'Definitely', 'It', 'There', 'also', 'black', 'come', 'delicious', 'desserts', 'dipping', 'flavors', 'great', 'sauce', 'soup', '', 'During', 'Saturday', 'Sunday', 'line', 'many', 'people', 'waiting', 'Food', 'lots', 'options', 'sauces', 'vegetables', '+', 'Good', 'Sa-ard', 'cream', 'good', 'ice', 'prices', 'restaurant', 'service', 'staff', 'Delicious', 'I', 'eat', 'every', 'go', 'time', 'less', 'little', 'quality', 'Waiting', 'branch', 'long', '25', 'Have', 'call', 'care', 'counting', 'day', 'food', 'hours', 'queue', 'queues', 'really', 'several', 'small', 'tastes', 'times', 'usual', 'waited', 'went', 'Fresh', 'Recommend', 'This', 'chili', 'eggs', 'fresh', 'garlic', 'meat', 'soy', 'sweet', 'tender', 'Other', 'enjoy', 'guest', 'meal', 'noisy', 'properly', 'talk', 'If', 'You', 'chance', 'hour', 'leave', 'let', 'number', 'wait', 'weekend', 'beef', 'bit', 'forgot', 'friends', 'hot', 'order', 'salty', 'waiters', '%.', '10', '30', '50', '?', 'Although', 'Bangna', 'Central', '

In [185]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

#TOPIC MODELING

In [186]:
num_topics = 4
chunksize = 4000
passes = 20
iterations = 50
eval_every = 1

temp = dictionary[0]
id2word = dictionary.id2token

model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                              alpha='auto', eta='auto', \
                              iterations=iterations, num_topics=num_topics, \
                              passes=passes, eval_every=eval_every)

In [187]:
vis = pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary, mds='mmds')
vis

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.085057 -0.072465       1        1  52.575151
0     -0.037165  0.113381       2        1  26.905105
1      0.077801 -0.074512       3        1  14.773918
3      0.044421  0.033595       4        1   5.745826, topic_info=         Term       Freq      Total Category  logprob  loglift
38          I   4.000000   4.000000  Default  30.0000  30.0000
35    service   8.000000   8.000000  Default  29.0000  29.0000
47     branch   4.000000   4.000000  Default  28.0000  28.0000
42       time   2.000000   2.000000  Default  27.0000  27.0000
31       good  11.000000  11.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
7   delicious   0.079760   7.504526   Topic4  -5.9045  -1.6875
8    desserts   0.079729   1.944780   Topic4  -5.9049  -0.3376
3       There   0.079727   1.681702   Topic4  -5.9049  -0.1923
4        also   0.079727   1.944768   Topic4  -5.9049  -0.3376
6        come   0.079726   2.630359   Topic4  -5.9049  -0.6396

[207 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
14        1  0.795541         
14        2  0.132590         
14        3  0.056824         
27        1  0.714847        +
49        1  0.714847       25
...     ...       ...      ...
92        1  0.994345     wait
21        1  0.691177  waiting
21        2  0.345588  waiting
193       3  0.880429     word
194       3  0.880419     yaki

[177 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2, 4])

#PREDICT TOPICS

In [188]:
model.show_topic(0)

[('', 0.051311336),
 ('good', 0.031288415),
 ('service', 0.01627083),
 ('branch', 0.016270822),
 ('queue', 0.016270047),
 ('staff', 0.016269507),
 ('management', 0.016268589),
 ('meat', 0.011265337),
 ('eat', 0.011263925),
 ('people', 0.011263384)]

In [189]:
model.show_topic(1)

[('', 0.045378346),
 ('delicious', 0.030850831),
 ('price', 0.02359067),
 ('like', 0.023588704),
 ('sauce', 0.016334897),
 ('good', 0.016334055),
 ('great', 0.01633282),
 ('flavors', 0.016332353),
 ('black', 0.016332066),
 ('There', 0.016331613)]

In [190]:
model.show_topic(2)

[('', 0.1579748),
 ('queue', 0.024943298),
 ('long', 0.0219191),
 ('food', 0.021918956),
 ('good', 0.018897798),
 ('staff', 0.0188971),
 ('service', 0.018895598),
 ('delicious', 0.015873292),
 ('quality', 0.015871866),
 ('Good', 0.0158717)]

In [191]:
model.show_topic(3)

[('I', 0.024522457),
 ('branch', 0.024522359),
 ('time', 0.02452231),
 ('good', 0.024522305),
 ('service', 0.024522291),
 ('great', 0.013625221),
 ('soup', 0.013625095),
 ('bit', 0.013623671),
 ('every', 0.013623648),
 ('friends', 0.0136236455)]

In [155]:
df['topics'] = df['Comment_token'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Comment_token'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [156]:
df

,ReviewID,Review,Comment_token,topics,score
0,cust01,It's very delicious. Definitely will come agai...,"It,delicious,Definitely,come,The,black,soup,gr...",1,0.991294
1,cust02,delicious !,delicious,0,0.046833
2,cust03,"During Saturday and Sunday, there are many peo...","During,Saturday,Sunday,,,many,people,waiting,line",1,0.983724
3,cust04,Food is great with lots of options in vegetabl...,"Food,great,lots,options,vegetables,sauces",1,0.975849
4,cust05,"Good service staff, Sa-ard restaurant, delicio...","Good,service,staff,,,Sa-ard,restaurant,,,delic...",1,0.994058
5,cust06,Delicious every time I go to eat The service s...,"Delicious,every,time,I,go,eat,The,service,staf...",2,0.986665
6,cust07,"Delicious, good quality, but a little less staff","Delicious,,,good,quality,,,little,less,staff",2,0.986665
7,cust08,Waiting for a long time at this branch,"Waiting,long,time,branch",0,0.013385
8,cust09,The food tastes delicious as usual. But the st...,"The,food,tastes,delicious,usual,But,staff,real...",2,0.996243
9,cust10,The meat here is very good. Fresh delicious an...,"The,meat,good,Fresh,delicious,good,service,Thi...",3,0.991906
